In [1]:
from typing import Tuple
import yaml
import os.path as osp

import igl
import trimesh
import numpy as np
import pyvista as pv
from matplotlib import pyplot as plt
from tqdm import tqdm
import pickle

import torch
import torch.nn.functional as F
from pytorch3d.ops import cot_laplacian

from preprocess import mesh_sampling_method
from dataset import MeshData
from models import VAE_coma


In [2]:
config_path = 'config/general_config.yaml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

# set the device, we can just assume we are using single GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = config["dataset"]
template = config["template"]
data_dir = osp.join('data', dataset)
template_fp = osp.join('template', template)

# get the up/down sampling matrix
ds_factor = config["ds_factor"]
edge_index_list, down_transform_list, up_transform_list = mesh_sampling_method(data_fp=data_dir,
                                                                                template_fp=template_fp,
                                                                                ds_factors=ds_factor,
                                                                                device=device)

# create the model
in_channels = config["model"]["in_channels"]
out_channels = config["model"]["out_channels"]
latent_channels = config["model"]["latent_channels"]
K = config["model"]["K"]

model = VAE_coma(in_channels = in_channels,
                out_channels = out_channels,
                latent_channels = latent_channels,
                edge_index = edge_index_list,
                down_transform = down_transform_list,
                up_transform = up_transform_list,
                K=K).to(device)

# get the mean and std of the CAESAR dataset(Traing set)
CAESAR_meshdata = MeshData(root=data_dir, template_fp=template_fp)
# of shape (10002, 3). is torch.Tensor
mean = CAESAR_meshdata.mean
std = CAESAR_meshdata.std

# load the template mesh
template_v, template_f = igl.read_triangle_mesh(template_fp)

# convert it into float32
# as in cot_laplacian, it only supports toch.float32. inv_areas = torch.zeros(V, dtype=torch.float32, device=verts.device)
template_v = template_v.astype('float32')
# convert it into a list of pytorch Tensor.
template_v = torch.from_numpy(template_v).cuda()
template_f = torch.from_numpy(template_f).cuda()

/home/wuliwuli/Desktop/predictor-net-measure/preprocess/mesh_sampling.py:59: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552411/work/torch/csrc/utils/tensor_new.cpp:230.)
  torch.LongTensor([spmat.tocoo().row,


Normalizing the training and testing dataset...
Normalization Done!


# Load PCA

In [3]:
pca = pickle.load(open("out/pca/pca.pkl", "rb"))

# The Laplacian function to compute the Laplacian for each mesh

In [4]:
def laplacian(vertex: torch.Tensor, faces: torch.Tensor, type: str) -> torch.Tensor:
    """
    Compute the laplacian of a mesh.
    Args:
        vertex: FloatTensor of shape (V, 3) giving vertex positions for V vertices.
        faces: LongTensor of shape (F, 3) giving faces.
        type: String giving the type of laplacian to compute. Must be either 'mean' or 'vertex'.
              mean: compute the mean laplacian value for all vertices.
              vertex: compute the laplacian value for each vertex.
    Returns:
        laplacian: FloatTensor of shape (V, 1) giving the laplacian matrix for the mesh.
                   OR, retuen the mean laplacian value for all vertices.
    """
    if type not in ['mean', 'vertex', 'vector']:
        raise ValueError('type must be one of mean, vertex or vector')

    # Compute the cotangent weights.
    L, inv_areas = cot_laplacian(vertex, faces)

    # NOTE: The diagonal entries of the cotangent weights matrix returned by the method are 0
    #       and need to be computed with the negative sum of the weights in each row.
    L_sum = torch.sparse.sum(L, dim=1).to_dense().view(-1, 1)

    # As shown in my thesis, the laplacian is given by: L_loss = L * M^{-1} * V, 
    # However, since the diagonal is zero, we modify it like: L_loss = (L - L_sum) * M^{-1} * V, 
    # where L is the cotangent weights matrix, 
    #       M is the mass matrix, with diagonal entries being 1/3 areas of the vertices,
    #       V is the vertex positions.
    
    # NOTE: I have no idea where the 0.25 comes from.
    #       In my opinion, it should be the weight should be 1/2 (cot alpha_{ij} + cot beta_{ij}).
    #       So, the coefficient should be 0.5. 
    mass_matrix_inv = 0.25 * inv_areas

    loss = (L.mm(vertex) - L_sum * vertex) * mass_matrix_inv

    laplacian_loss = torch.norm(loss, dim=1)

    if type == 'mean':
        return torch.mean(laplacian_loss)
    elif type == 'vertex':
        return laplacian_loss
    elif type == 'vector':
        return loss


In [5]:
def get_lap(sample_vals: np.ndarray, mean: torch.Tensor, std: torch.Tensor, faces: torch.Tensor, model):
    """_summary_
        load the model from model_path
        sample several points from the latent space
        for each corresponding mesh, return the laplacian loss(type = 'vertex')

    Args:
        sample_vals (np.ndarray): of shape (n_samples, ). We would sample n_samples from the latent space. And use the same vals for all 8 dimensions.
        mean (torch.Tensor): of shape (10002, 3). is torch.Tensor
        std (torch.Tensor): of shape (10002, 3). is torch.Tensor
        faces (torch.Tensor): of shape (10002, 3). The faces of the template mesh.
        model (VAE_coma): the model
    """

    pca = model

    # Sample from the latent space
    # create a tensor of shape (10002, n_samples, 6)
    # 10002 is the number of vertices of the template mesh, 6 is the number of latent dimensions
    lap_vals = np.zeros((10002, sample_vals.shape[0], 6))
    # M_matrix is of shape (8, 7)
    M_matrix = np.load("out/pca/M.npy")
    
    # for each latent dimension
    for i in tqdm(range(6)):
        # 7 = 6 + 1. 6 is the number of latent dimensions, we add one more bias term after that.
        latent_val = np.zeros((sample_vals.shape[0], 7))

        latent_val[:, i] = sample_vals
        
        # now the new_w is of shape (n_samples, 8)
        new_w = np.matmul(M_matrix, latent_val.T).T
                
        # pca.inverse_transform() will return the mesh in the original space
        v = pca.inverse_transform(new_w).reshape(-1, 10002, 3)    
        v = v.astype('float32')
        v = (torch.from_numpy(v) + mean).cuda()

        for j in range(sample_vals.shape[0]):
            # get the laplacian loss for each mesh
            laplacian_loss = laplacian(v[j], faces, type='vertex')
            lap_vals[:, j, i] = laplacian_loss.cpu().numpy()

    return lap_vals

# The Laplacian Loss on template mesh

In [6]:
temp_lap_loss_mean = laplacian(template_v, template_f, 'mean')
print(f"The mean loss is {temp_lap_loss_mean:.3f}")

temp_lap_loss_vertex = laplacian(template_v, template_f, 'vertex')
print("The max loss is {:.3f}".format(torch.max(temp_lap_loss_vertex).item()))
print("The median loss is {:.3f}".format(torch.median(temp_lap_loss_vertex).item()))
print("The std loss is {:.3f}".format(torch.std(temp_lap_loss_vertex).item()))

# plt.hist(temp_lap_loss_vertex .cpu().numpy(), bins=100)

# temp_v, temp_f = igl.read_triangle_mesh(template_fp)
# temp_mesh = trimesh.Trimesh(vertices=temp_v, faces=temp_f)
# vertex_normals = trimesh.smoothing.get_vertices_normals(temp_mesh)
# temp_lap_loss_vector = laplacian(template_v, template_f, 'vector').cpu().numpy()

# lap = np.sum(temp_lap_loss_vector * vertex_normals, axis=1)
# temp_mesh = pv.wrap(temp_mesh)
# temp_mesh["lap_loss"] = lap
# temp_mesh.save(osp.join('result', 'laplacian', 'template.vtk'))

# plt.hist(lap, bins=100)

# print("The max loss is {:.3f}".format(np.max(lap)))
# print("The min loss is {:.3f}".format(np.min(lap)))
# print("The median loss is {:.3f}".format(np.median(lap)))
# print("The std loss is {:.3f}".format(np.std(lap)))


The mean loss is 12.199
The max loss is 1121.988
The median loss is 4.100
The std loss is 28.450


In [7]:
temp_v, temp_f = igl.read_triangle_mesh(template_fp)
temp_mesh = trimesh.Trimesh(vertices=temp_v, faces=temp_f)
temp_lap_loss_vertex = laplacian(template_v, template_f, 'vertex').cpu().numpy()

temp_mesh = pv.wrap(temp_mesh)
temp_mesh["lap_loss"] = temp_lap_loss_vertex
temp_mesh_diff = np.abs(temp_lap_loss_vertex - temp_lap_loss_vertex)
temp_mesh["temp_diff"] = temp_mesh_diff
temp_mesh.save(osp.join('result', 'laplacian', 'template_mean.vtk'))

# PCA on Laplacian

In [8]:
sample_vals = np.linspace(-1, 1, 50)
lap_loss = get_lap(sample_vals, mean, std, template_f, pca)

# write the printed info into a file
with open("result/pca/laplacian.txt", "w") as f:
    print("The mean loss for our method is {:.3f}".format(np.mean(lap_loss)), file=f)
    print("The max loss for our method is {:.3f}".format(np.max(lap_loss)), file=f)
    print("The median loss for our method is {:.3f}".format(np.median(lap_loss)), file=f)
    print("The std loss for our method is {:.3f}".format(np.std(lap_loss)), file=f)

100%|██████████| 6/6 [00:00<00:00, 12.97it/s]


In [9]:
latent_vector = np.zeros((1, 7))
for i in range(6):
    latent_vector[:, i] = 0.1
    
M_matrix = np.load("out/pca/M.npy")
new_w = np.matmul(M_matrix, latent_vector.T).T
v = pca.inverse_transform(new_w).reshape(10002, 3)
v = v.astype('float32')
v = torch.from_numpy(v)
v = (v + mean).numpy()

temp_v, temp_f = igl.read_triangle_mesh(template_fp)
pca_mesh = trimesh.Trimesh(vertices=v, faces=temp_f)

print(pca_mesh.vertices.shape)

v = torch.from_numpy(v.astype('float32')).cuda()

pca_lap_loss_vertex = laplacian(v, template_f, 'vertex').cpu().numpy()

print(pca_lap_loss_vertex.shape)
print(temp_lap_loss_vertex.shape)

pca_lap_loss_diff = np.abs(pca_lap_loss_vertex - temp_lap_loss_vertex)
pca_mean_mesh = pv.wrap(pca_mesh)
pca_mean_mesh["lap_loss"] = pca_lap_loss_vertex
pca_mean_mesh["lap_loss_diff"] = pca_lap_loss_diff
pca_mean_mesh.save(osp.join('result', 'pca', 'pca_mean.vtk'))

(10002, 3)
(10002,)
(10002,)


In [10]:
# model.load_state_dict(torch.load("out/vae_coma/trainer12/20231220-123538/model.pth"))
# latent_vector = torch.zeros((1, 8)).cuda()
# v = decode_func(latent_vector, model)
# # convert to numpy array
# v = v.detach().cpu()
# # denormalize the vertices
# v = v * std + mean
# v = v.reshape(-1, 3)

# temp_v, temp_f = igl.read_triangle_mesh(template_fp)
# our_method_mesh = trimesh.Trimesh(vertices=v, faces=temp_f)
# v = v.cuda()
# our_method_lap_loss_vertex = laplacian(v, template_f, 'vertex').cpu().numpy()
# our_method_lap_loss_diff = np.abs(our_method_lap_loss_vertex - temp_lap_loss_vertex)
# our_method_mean_mesh = pv.wrap(our_method_mesh)
# our_method_mean_mesh["lap_loss"] = our_method_lap_loss_vertex
# our_method_mean_mesh["lap_loss_diff"] = our_method_lap_loss_diff
# our_method_mean_mesh.save(osp.join('result', 'laplacian', 'our_method_mean.vtk'))